Changes:
* All categoricals as embedding

Things to add :
1. Add categoricals
    * First as one encoding
    * Second as vectorized version based on other variables like price
    * Third try to do also encoding of the target (Beware of cross validation)
2. Add dense features (This is where to put encodings)
    * Dont forget to scale (with train+test)
3. Add title features
4. Add Regularization !
5. Add Emoji detection (character tokenizer)
6. Use desc/title features as extra variables in a tree
6. Add extra features found in other notebooks

Don't forget to normalize.
Try SnowballStemmer in Russian

CNN 10 epochs:
* desc : 0.235
* desc + region + pcn : 0.233
* desc + region + pcn + cn : 0.231
* desc + region + pcn + cn + ut : 0.230
* desc + region + pcn + cn + ut + title : 0.227
* desc + region + pcn + cn + ut + title + image_top + city : 0.224
* desc + region + pcn + cn + ut + title + image_top + city + logprice + price_char : 0.223
* desc + region + pcn + cn + ut + title + image_top + city + logprice + price_char + Params : 0.221
* desc + region + pcn + cn + ut + title + image_top + city + logprice + price_char + Params + User_id + Weekday : 0.2206
* ###1### = desc + region + pcn + cn + ut + title + image_top + city + logprice + price_char + Params + User_id + Param_1 + logItem_seq : 0.2194
* ###1### + Encodings : 

In [1]:
import pandas as pd
import numpy as np
import keras
from keras_tqdm import TQDMNotebookCallback
import gensim
import re
from tqdm import tqdm,tqdm_notebook,tqdm_pandas
import gc

from nltk.corpus import stopwords
from nltk.stem.snowball import RussianStemmer
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler

E:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
E:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
DATA_PATH = 'E:/Kaggle/Avito/'
REMOVE_NA_DESC = False

In [3]:
num_words = 100000
max_len = 150
seed = 32

## Load word2vec dictionary

In [4]:
word2vec = gensim.models.Word2Vec.load(DATA_PATH+'avito300_sg.w2v')

## Load Data

In [5]:
train = pd.read_csv(DATA_PATH+'train.csv')
test = pd.read_csv(DATA_PATH+'test.csv')
if REMOVE_NA_DESC:
    train=train[train['description'].notnull()]

In [6]:
complete_data = pd.concat([train,test],axis=0,ignore_index=True)

## Feature Engineering

In [7]:
complete_data['params']=complete_data['param_1'].astype(str)+complete_data['param_2'].astype(str)+complete_data['param_3'].astype(str)
for col_text in ['params','title','description']:
    complete_data[col_text].fillna('thisismissing',inplace=True)
    complete_data[col_text] = complete_data[col_text].astype(str)

#### Categoricals as One Hot
* Need to deal with missing values better.
* Deal with low frequency categories and categories not in test set. Either impute it by something close that exists in train or by a new class
* Try form of embedding instead

In [8]:
categoricals_embs = ['parent_category_name','category_name','param_1']

In [9]:
catembs = []
le=LabelEncoder()
for cat in categoricals_embs: #Must deal with cities not in train set
    complete_data[cat].fillna(-999,inplace=True)
    complete_data[cat] = le.fit_transform(complete_data[cat].astype(str))
    catembs.append(complete_data[cat].max()+1)
X_categoricals = complete_data[categoricals_embs].values.astype('float32')

In [10]:
print(catembs) 

[9, 47, 372]


#### Text stuff

In [11]:
english_stopwords = set(stopwords.words('english'))
russian_stopwords = set(stopwords.words('russian'))
russian_stopwords = russian_stopwords.union(english_stopwords)
def preprocess(x,stop_words=None):
    x = keras.preprocessing.text.text_to_word_sequence(x)
    if stop_words:
        return [word for word in x if word not in russian_stopwords]
    else:
        return x

In [12]:
def make_token_matrix(data,text_col,num_words,max_len,stop_words=None):
    
    print('Create Tokenizer...',end=' ')
    
    texts = data[text_col].astype(str)
    tokenizer = keras.preprocessing.text.Tokenizer(num_words=num_words,lower=True)
    tokenizer.fit_on_texts(texts)
    
    print('Preprocess Text...',end=' ')
    texts = texts.apply(lambda x: preprocess(x,stop_words))
    
    print('Create Matrix...',end=' ')
    X = tokenizer.texts_to_sequences(texts)
    X = keras.preprocessing.sequence.pad_sequences(X,padding='pre',truncating='post',maxlen=max_len)
    
    print('Done !')
    return X,tokenizer

In [13]:
complete_data['description'] = complete_data['description'].astype(str)

In [14]:
X_desc, tokenizer_desc = make_token_matrix(complete_data,'description',num_words,max_len,stop_words=russian_stopwords)
word2idx = tokenizer_desc.word_index

Create Tokenizer... Preprocess Text... Create Matrix... Done !


In [15]:
word2idx = tokenizer_desc.word_index
idx2word = {i:w for w,i in word2idx.items()}

In [16]:
for i in X_desc[0]:
    if i!=0:
        print(idx2word[i],end=' ')
complete_data.iloc[0]['description']

кокон сна малыша пользовались меньше месяца цвет серый 

'Кокон для сна малыша,пользовались меньше месяца.цвет серый'

In [17]:
X_title, tokenizer_title = make_token_matrix(complete_data,'title',num_words,max_len,stop_words=russian_stopwords)    
word2idx_title = tokenizer_title.word_index

Create Tokenizer... Preprocess Text... Create Matrix... Done !


In [18]:
params_len = complete_data['params'].str.len().max()
X_params, tokenizer_params = make_token_matrix(complete_data,'params',num_words,params_len)
word2idx_params = tokenizer_params.word_index

Create Tokenizer... Preprocess Text... Create Matrix... Done !


In [19]:
print(params_len)

63


## Concatenate and Splits

In [20]:
slices = [0, 
          max_len, #Description
          max_len+max_len, #Title
          max_len+max_len+params_len, #Params
          *[max_len+max_len+params_len+i+1 for i in range(len(catembs))]] #Categoricals as embeding layer 

slices_bounds = [(slices[i],slices[i+1]) for i,s in enumerate(slices) if i<len(slices)-1]

In [21]:
slices_bounds

[(0, 150), (150, 300), (300, 363), (363, 364), (364, 365), (365, 366)]

In [22]:
X = np.concatenate([X_desc,X_title,X_params,X_categoricals],axis=1)
#del X_title
#del X_params
#del X_dense
#del X_price
#del X_desc
#del X_categoricals

In [23]:
test_index = complete_data['image_top_1'].isna()
train_index = ~test_index

X_tr = X[train_index].astype('float32')
X_board = X[test_index].astype('float32')

y = complete_data[train_index]['image_top_1']

## Make model

In [24]:
def make_pretrain_embedding(w2idx,word2vec,embed_dim,num_words):
    unknown_words = []
    embeddings = np.zeros((num_words+1,embed_dim))  #0 is a special token
    for word,idx in w2idx.items(): #starts at 1    
        if idx>num_words:
            break 
        try:
            vect = word2vec[word]
            embeddings[idx]=vect/np.linalg.norm(vect)
        except KeyError:
            unknown_words.append(word)
    print('Number of words with no embeddings',len(unknown_words))
    
    return embeddings, unknown_words

In [25]:
pretrained_desc, unknown_words1 = make_pretrain_embedding(word2idx,word2vec,300,num_words)
pretrained_title, unknown_words2 = make_pretrain_embedding(word2idx_title,word2vec,300,num_words)
#pretrained_params = make_pretrain_embedding(word2idx_params,word2vec,300,num_words)

E:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


Number of words with no embeddings 211
Number of words with no embeddings 7831


In [26]:
y.nunique()

3063

In [27]:
def make_cnn_model(desc_len,
                   params_len,
                   catembs,
                   embed_dim,
                   pretrained_desc,pretrained_title,#pretrained_params,
                   dropout=0,
                   trainable_embeddings=False,
                   conv_size=128):
    
    desc_input = keras.layers.Input(shape=(desc_len,))
    title_input = keras.layers.Input(shape=(desc_len,))
    params_input = keras.layers.Input(shape=(params_len,))
    
    #Description part
    embedded = keras.layers.Embedding(input_dim=pretrained_desc.shape[0],
                                      output_dim=embed_dim,
                                      input_length=max_len,
                                      weights=[pretrained_desc],
                                      trainable=trainable_embeddings)(desc_input)
    
    embedded = keras.layers.SpatialDropout1D(dropout+0.1)(embedded)
    X_1 = keras.layers.Conv1D(conv_size,kernel_size=1,activation='relu')(embedded)
    X_1 = keras.layers.GlobalMaxPooling1D()(X_1)
    X_2 = keras.layers.Conv1D(conv_size,kernel_size=3,activation='relu')(embedded)
    X_2 = keras.layers.GlobalMaxPooling1D()(X_2)
    X_3 = keras.layers.Conv1D(conv_size,kernel_size=5,activation='relu')(embedded)
    X_3 = keras.layers.GlobalMaxPooling1D()(X_3)
    
    desc_features = keras.layers.Concatenate()([X_1,X_2,X_3])
    desc_features = keras.layers.Dropout(dropout+0.1)(desc_features)
    desc_features = keras.layers.Dense(32,activation='relu')(desc_features)
    
    #Title
    embedded_title = keras.layers.Embedding(input_dim=pretrained_title.shape[0],
                                      output_dim=embed_dim,
                                      input_length=max_len,
                                      weights=[pretrained_title],
                                      trainable=trainable_embeddings)(title_input)
    
    embedded_title = keras.layers.SpatialDropout1D(dropout+0.1)(embedded_title)
    Z_1 = keras.layers.Conv1D(conv_size,kernel_size=1,activation='relu')(embedded_title)
    Z_1 = keras.layers.GlobalMaxPooling1D()(Z_1)
    Z_2 = keras.layers.Conv1D(conv_size,kernel_size=3,activation='relu')(embedded_title)
    Z_2 = keras.layers.GlobalMaxPooling1D()(Z_2)
    Z_3 = keras.layers.Conv1D(conv_size,kernel_size=5,activation='relu')(embedded_title)
    Z_3 = keras.layers.GlobalMaxPooling1D()(Z_3)
    
    title_features = keras.layers.Concatenate()([Z_1,Z_2,Z_3])
    title_features = keras.layers.Dropout(dropout+0.1)(title_features)
    title_features = keras.layers.Dense(32,activation='relu')(title_features)
    
    #Params
    embedded_params = keras.layers.Embedding(input_dim=len(tokenizer_params.word_index)+1,
                                      output_dim=100,
                                      input_length=params_len,trainable=True)(params_input)
    params_features = keras.layers.CuDNNGRU(32,return_sequences=True)(embedded_params)
    params_avg = keras.layers.GlobalAveragePooling1D()(params_features)
    params_max = keras.layers.GlobalMaxPooling1D()(params_features)
    params_features = keras.layers.Concatenate()([params_avg,params_max])
    
    #Categoricals    
    cat_embs_inputs = []
    cat_embs_embeded = []
    for i in range(len(catembs)):
        cat_embs_inputs.append(keras.layers.Input(shape=(1,)))
        cat_embs_embeded.append(keras.layers.Embedding(input_dim=catembs[i],
                                                      output_dim=16,
                                                      input_length=1,trainable=True)(cat_embs_inputs[i]))
    
    cat_emb_features = keras.layers.Concatenate()(cat_embs_embeded)
    cat_emb_features = keras.layers.Flatten()(cat_emb_features)
    cat_emb_features = keras.layers.Dropout(dropout)(cat_emb_features)
    cat_emb_features = keras.layers.Dense(128,activation='relu')(cat_emb_features)
    cat_emb_features = keras.layers.Dropout(dropout)(cat_emb_features)
    cat_emb_features = keras.layers.Dense(32,activation='relu')(cat_emb_features)
    
    #Concatenate Features
    X = keras.layers.Concatenate()([cat_emb_features,desc_features,title_features,params_features])
    X = keras.layers.Dropout(dropout+0.1)(X)
    X = keras.layers.Dense(256,activation='relu')(X)
    X = keras.layers.Dropout(dropout)(X)
    X = keras.layers.Dense(128,activation='relu')(X)
    X = keras.layers.BatchNormalization()(X)
    
    y_hat = keras.layers.Dense(3067,activation='softmax')(X)
    
    cnn_model = keras.Model(inputs=[desc_input,title_input,params_input,*cat_embs_inputs],outputs=y_hat)
    
    return cnn_model

In [28]:
gc.collect()
np.random.seed(seed)
VALID=False
if VALID:
    X_tr_tr, X_val, y_tr, y_val = train_test_split(X_tr,y,test_size=0.1,random_state=seed)
    keras.backend.clear_session() #Reset   
    cnn_model = make_cnn_model(max_len,
                               params_len,
                               catembs,
                               300,
                               pretrained_desc,pretrained_title,
                               dropout=0.2,
                               trainable_embeddings=False, conv_size=128) #If allows train of embedding words, will have to restrict vocab to the train set
    Adam = keras.optimizers.Adam(0.001)
    cnn_model.compile(optimizer=Adam,loss=keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
    epochs = 10 #Beware of overfit
    cnn_model.fit([X_tr_tr[:,s[0]:s[1]] for s in slices_bounds],
              y_tr,
              validation_data=([X_val[:,s[0]:s[1]] for s in slices_bounds], y_val),
              batch_size=512,epochs=epochs,verbose = 0,callbacks=[TQDMNotebookCallback(leave_inner=True)],)
    predictions = cnn_model.predict([X_val[:,s[0]:s[1]] for s in slices_bounds]).flatten()
    r = np.sqrt(np.mean((predictions.clip(0,1)-y_val)**2))
    print(f'RMSE : {r:.4f}')
    
else:
    cnn_model = make_cnn_model(max_len,
                           params_len,
                           catembs,
                           300,
                           pretrained_desc,pretrained_title,
                           dropout=0.2,
                           trainable_embeddings=False, conv_size=128) #If allows train of embedding words, will have to restrict vocab to the train set
    Adam = keras.optimizers.Adam(0.001)
    cnn_model.compile(optimizer=Adam,loss=keras.losses.sparse_categorical_crossentropy,metrics=['accuracy'])
    epochs = 10 #Beware of overfit
    cnn_model.fit([X_tr[:,s[0]:s[1]] for s in slices_bounds],
              y,
              batch_size=512,epochs=epochs,verbose = 0,callbacks=[TQDMNotebookCallback(leave_inner=True)],)

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [50]:
guess_image = complete_data[['item_id']].copy()
guess_image['Prob_image_top1'] = 0
guess_image['Guess_image_top1'] = 0
guess_image['Guess_image_top2'] = 0
guess_image['Guess_image_top3'] = 0

In [51]:
for k in range(41):
    X_batch = X[50000*k:50000*(k+1),:]
    predictions_image = cnn_model.predict([X_batch[:,s[0]:s[1]] for s in slices_bounds])
    guess_image.iloc[50000*k:50000*(k+1),1]=predictions_image.max(axis=1)
    guess_image.iloc[50000*k:50000*(k+1),2]=predictions_image.argmax(axis=1)
    predictions_image = predictions_image.argsort(axis=1)
    guess_image.iloc[50000*k:50000*(k+1),3]=predictions_image[:,-2:-1]
    guess_image.iloc[50000*k:50000*(k+1),4]=predictions_image[:,-3:-2]

In [53]:
guess_image.to_csv(DATA_PATH+'image_guess.csv',index=False)

In [54]:
guess_image['real_image_top']=complete_data['image_top_1']

In [55]:
guess_image

,item_id,Prob_image_top1,Guess_image_top1,Guess_image_top2,Guess_image_top3,real_image_top
0,b912c3c6a6ad,0.805290,1008,1009,75,1008.0
1,2dac0150717d,0.403171,1522,1496,1489,692.0
2,ba83aefab5dc,0.670797,3042,3039,3040,3032.0
3,02996f1dd2ea,0.903954,796,1133,1442,796.0
4,7c90be56d2ab,0.414640,1108,2264,2262,2264.0
5,51e0962387f7,0.954151,796,1133,999,796.0
6,c4f260a2b48a,0.761278,1400,1383,1197,2823.0
7,6b71309d6a8a,0.584889,567,658,580,567.0
8,c5b969cb63a2,0.145360,404,572,399,415.0
9,b1570962e68c,0.583316,88,85,86,46.0
